In [1]:
import pandas as pd
import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '../../secret.json',
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

path = 'Copy of Hungerstation - Keeta Competitive Data Extract (01.11.2024) - Sheet4.csv'

In [2]:
df = pd.read_csv(path)

In [3]:
df.head()

,Global Entity\,Logistics Area,Source,SF Status Raw,Vendor name,Vendor Grade,City,Address,Latitude,Longitude,Phone,Main Cuisine,Lead ID,Accuracy,Status,Handler email,GRID
0,HS_SA,logistics,keeta,No Match,Akbar Al Bukhari Restaurant,NaN,Riyadh,"Badr, Al Imam Malik Rd, Riyadh 14724",24.5486,46.7070,NaN,"Arabic, Saudi",453255,Invalid lead (Add reason),Already exist,Bandar Alshathri,HA7S0F
1,HS_SA,logistics_proximity,keeta,No Match,Al Baraka Bukhari Restaurant,NaN,Riyadh,"Arafat Rd، 4451 7244 فتح مكة, Riyadh 14724",24.5402,46.7132,NaN,"Arabic, Saudi, Meat, Bokhari",469890,Invalid lead (Add reason),Already exist,Ahmed khalil,HRBMVS
2,HS_SA,logistics_proximity,keeta,No Match,Al Zalal Restaurant,NaN,Riyadh,"3751 7503 Prince Mamduh Bin Abdulaziz St, As S...",24.7101,46.6926,9.661128e+11,"Asian, Pakistani, Fine Dining, Meat, Local Fla...",489464,Invalid lead (Add reason),Duplicate,Mohammad Nasreddin,NaN
3,HS_SA,logistics,keeta,No Match,Alam Althura Markets Almajd,NaN,Riyadh,"4210 Eastern Ring Branch Rd, Ar Rabwah, 6838, ...",24.6799,46.7761,NaN,"Dessert, Snacks",492730,Invalid lead (Add reason),Already exist,Mohammad Nasreddin,4CUZPO
4,HS_SA,sales,keeta,No Match,Alstuor,NaN,Riyadh,"JQ33+JG7, Al Mansourah, Riyadh 12694, Saudi Ar...",24.6041,46.7538,NaN,"Fast Food, Shawarma, Meat",580340,Invalid lead (Add reason),Already exist,Bandar Alshathri,4CJ71O


In [4]:
grids = df['GRID'].unique().tolist()

In [5]:
len(grids)

52

In [6]:
grids

['HA7S0F',
 'HRBMVS',
 nan,
 '4CUZPO',
 '4CJ71O',
 'HZXYHI',
 'HAGA7N',
 'HR30VK',
 'HA9BU6',
 'HADJAO',
 'HAWRTB',
 '4FBZ8Y',
 'HWEF7E',
 '4CMYE7',
 'HREL4Z',
 'HWIUFY',
 '4CCFDW',
 'HA7C6L',
 '4FBU6H',
 'HAXGW9',
 'HWSJHO',
 'HR8AX1',
 '4CX2A7',
 'HWUF53',
 'HWV6OH',
 '4CJE79',
 'HREK1A',
 'HADY57',
 '4FB7NC',
 'HWX5AU',
 'HRE5M1',
 'HANHE1',
 '4C0VH1',
 '4FI7GQ',
 '4FBKZC',
 '4F668U',
 'HAP8ZY',
 'HAMMCB',
 '4CJEEF',
 'HR6N3L',
 '4F6O1P',
 '4FBM5R',
 '4C48UL',
 '4FB0YG',
 'HA0YB4',
 'HAW09D',
 '4F2DKL',
 'HRHDOQ',
 '4COBEU',
 'HRG69U',
 '4C2QOH',
 'HR9N3L']

In [7]:
# Remove nan
len([grid for grid in grids if isinstance(grid,str)])

51

In [8]:
grid_list_query = "('" + "', '".join(str(grid) for grid in grids if isinstance(grid,str)) + "')"

In [9]:
grid_list_query

"('HA7S0F', 'HRBMVS', '4CUZPO', '4CJ71O', 'HZXYHI', 'HAGA7N', 'HR30VK', 'HA9BU6', 'HADJAO', 'HAWRTB', '4FBZ8Y', 'HWEF7E', '4CMYE7', 'HREL4Z', 'HWIUFY', '4CCFDW', 'HA7C6L', '4FBU6H', 'HAXGW9', 'HWSJHO', 'HR8AX1', '4CX2A7', 'HWUF53', 'HWV6OH', '4CJE79', 'HREK1A', 'HADY57', '4FB7NC', 'HWX5AU', 'HRE5M1', 'HANHE1', '4C0VH1', '4FI7GQ', '4FBKZC', '4F668U', 'HAP8ZY', 'HAMMCB', '4CJEEF', 'HR6N3L', '4F6O1P', '4FBM5R', '4C48UL', '4FB0YG', 'HA0YB4', 'HAW09D', '4F2DKL', 'HRHDOQ', '4COBEU', 'HRG69U', '4C2QOH', 'HR9N3L')"

In [27]:
# Run a query
query = f"""
select 
global_entity_id, 
lead_id,
lead_source,
restaurant_name,
restaurant_area,
restaurant_address,
restaurant_post_code,
restaurant_lat,
restaurant_long,
restaurant_city
from `dh-global-sales-data.leadgen_cl.vendor_complete`
where lead_id in {grid_list_query}
"""

df_sf_data = pandas_gbq.read_gbq(query, project_id=credentials.project_id, credentials=credentials)

/opt/homebrew/anaconda3/envs/vm_analysis/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [28]:
df_merged = df.merge(df_sf_data, how='left', left_on='GRID', right_on='lead_id')

In [29]:
df_merged

,Global Entity\,Logistics Area,Source,SF Status Raw,Vendor name,Vendor Grade,City,Address,Latitude,Longitude,...,global_entity_id,lead_id,lead_source,restaurant_name,restaurant_area,restaurant_address,restaurant_post_code,restaurant_lat,restaurant_long,restaurant_city
0,HS_SA,logistics,keeta,No Match,Akbar Al Bukhari Restaurant,NaN,Riyadh,"Badr, Al Imam Malik Rd, Riyadh 14724",24.5486,46.7070,...,HS_SA,HA7S0F,salesforce,Rukn Akbar Al Bukhari Restaurants: An Nasim Al...,An Nasim Al Gharbi,Omar Bin Abdullah ST,14234,24.722327,46.835070,Riyadh
1,HS_SA,logistics_proximity,keeta,No Match,Al Baraka Bukhari Restaurant,NaN,Riyadh,"Arafat Rd، 4451 7244 فتح مكة, Riyadh 14724",24.5402,46.7132,...,HS_SA,HRBMVS,salesforce,Dhawq qand : Al Marwah,Al Marwah,"6806 Al Khalil Ibn Ahmad, Al Marwah, 3790, Riy...",14721,24.546749,46.688294,Riyadh
2,HS_SA,logistics_proximity,keeta,No Match,Al Zalal Restaurant,NaN,Riyadh,"3751 7503 Prince Mamduh Bin Abdulaziz St, As S...",24.7101,46.6926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HS_SA,logistics,keeta,No Match,Alam Althura Markets Almajd,NaN,Riyadh,"4210 Eastern Ring Branch Rd, Ar Rabwah, 6838, ...",24.6799,46.7761,...,HS_SA,4CUZPO,salesforce,Corn World,As Safraa,"26.36760976481661, 43.96233186537955",None,26.367427,43.962475,Buraydah
4,HS_SA,sales,keeta,No Match,Alstuor,NaN,Riyadh,"JQ33+JG7, Al Mansourah, Riyadh 12694, Saudi Ar...",24.6041,46.7538,...,HS_SA,4CJ71O,salesforce,"Cleaver , Al Mansourah",Al Mansourah,"24.604185707693016, 46.753782342328556",None,24.604063,46.753813,Riyadh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,HS_SA,sales,keeta,No Match,تراينقل سويت,NaN,Riyadh,"3001 Al Souweidi Al Aam, As Suwaidi, Riyadh 12794",24.5870,46.6679,...,HS_SA,4COBEU,salesforce,Triangle Sweet,Masiaf,"24.763962629927736, 46.6805432",None,24.763909,46.680581,Riyadh
63,HS_SA,sales,keeta,No Match,تكالافا,NaN,Al-Kharj,"Mosque, An Nuzhah, Al-Kharj, Saudi Arabia",24.1608,47.3461,...,HS_SA,HRG69U,salesforce,Tikkalava : An Nuzhah,An Nuzhah,طريق عمر بن عبدالعزيز,11932,24.160817,47.346120,Al Kharj
64,HS_SA,logistics,keeta,No Match,ذا رد باستا كَب,NaN,Riyadh,RBBA4438، 4438، 7062 Al Madina Al Munawwarah R...,24.5978,46.6388,...,HS_SA,4C2QOH,salesforce,The Red Pasta Cup : Al Manar,Al Manar,"26.356453269947043, 43.934073821953476",None,26.356409,43.934086,Buraydah
65,HS_SA,sales,keeta,No Match,ذا رد باستا كَب,NaN,Riyadh,"QP68+C8R, Saad Ibn Ayyub, An Nuzhah, Riyadh 12...",24.7611,46.7158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
import math

def haversine(lat1, lon1, lat2, lon2) -> float:
    """
    Returns haversine distance in *km*
    """
    # Convert coordinates from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))

    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

In [31]:
df_merged["haversine_distance"]= df_merged.apply(lambda x: haversine(x.Latitude, x.Longitude, x.restaurant_lat, x.restaurant_long), axis=1)

In [35]:
df_merged.to_csv('data_with_sf_and_haversine.csv')

In [42]:
haversine(24.5978,46.6388,24.7611,46.7158)

19.754622517195656